In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.1.0


In [2]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

In [4]:
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32))
print(cube_func_int32)

print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1,2,3])))

True
True


In [5]:
cube_func_int32.graph

In [6]:
print(cube(tf.constant([1, 2, 3])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [8]:
# 将带signature的function保存成savedmodel
# 1、建立一个module
# 2、给定义好的module加一个成员变量
# 3、将module保存成savedmodel
to_export = tf.Module()
to_export.cube = cube
tf.saved_model.save(to_export, "./signature_to_saved_model")

INFO:tensorflow:Assets written to: ./signature_to_saved_model/assets


In [10]:
!saved_model_cli show --dir ./signature_to_saved_model --all

2020-02-28 00:58:36.523419: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-02-28 00:58:36.523539: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-02-28 00:58:36.523560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_

In [11]:
imported = tf.saved_model.load("./signature_to_saved_model")
imported.cube(tf.constant([1, 2, 3]))

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1,  8, 27], dtype=int32)>